In [1]:
import os
import librosa
import numpy as np
import time
import joblib
from sklearn.metrics import accuracy_score
import librosa.display
import scipy.fftpack as fftpack
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'librosa'

In [3]:
train_path = 'E:/Queenless/archive/nuhive_processed/train'
val_path = 'E:/Queenless/archive/nuhive_processed/val'
test_path = 'E:/Queenless/archive/nuhive_processed/test'

output_dir = 'E:/Queenless/fft_features'
os.makedirs(output_dir, exist_ok=True)

In [3]:
def pre_emphasis(signal_in, pre_emph=0.97):
    """
    Áp dụng pre-emphasis để nhấn mạnh các tần số cao.
    
    Args:
        signal_in: Tín hiệu đầu vào
        pre_emph: Hệ số pre-emphasis, mặc định là 0.97
        
    Returns:
        Tín hiệu sau khi áp dụng pre-emphasis
    """
    return np.append(signal_in[0], signal_in[1:] - pre_emph * signal_in[:-1])

def framing(signal_in, sample_rate, frame_size=0.025, hop_length=256, overlap=0.6):
    """
    Chia tín hiệu thành các frame với kích thước và bước nhảy xác định.
    
    Args:
        signal_in: Tín hiệu đầu vào
        sample_rate: Tần số lấy mẫu
        frame_size: Kích thước frame (giây hoặc số mẫu nếu frame_size > 1)
        hop_length: Bước nhảy (giây hoặc số mẫu nếu hop_length > 1)
                    Nếu None, sẽ được tính dựa trên overlap
        overlap: Tỷ lệ chồng lấp giữa các frame, mặc định là 0.6 (60%)
                Chỉ được sử dụng khi hop_length là None
        
    Returns:
        Mảng 2D chứa các frame
    """
    # Xác định kích thước frame trong số mẫu
    if frame_size <= 1:  # Nếu frame_size <= 1, xem như đơn vị là giây
        frame_length = int(round(frame_size * sample_rate))
    else:  # Ngược lại, xem như đơn vị là số mẫu
        frame_length = int(frame_size)
    
    # Xác định hop_length (bước nhảy) trong số mẫu
    if hop_length is None:
        # Tính hop_length từ overlap
        frame_step = int(frame_length * (1 - overlap))
    elif hop_length <= 1:  # Nếu hop_length <= 1, xem như đơn vị là giây
        frame_step = int(round(hop_length * sample_rate))
    else:  # Ngược lại, xem như đơn vị là số mẫu
        frame_step = int(hop_length)
    
    # Đảm bảo hop_length không quá nhỏ
    frame_step = max(1, frame_step)
    
    signal_length = len(signal_in)
    
    # Tính số frame cần thiết
    num_frames = int(np.ceil((signal_length - frame_length) / frame_step)) + 1
    
    # Đệm tín hiệu để đảm bảo có đủ mẫu cho tất cả các frame
    pad_signal_length = (num_frames - 1) * frame_step + frame_length
    pad_zeros = np.zeros(pad_signal_length - signal_length)
    padded_signal = np.concatenate([signal_in, pad_zeros])
    
    # Tạo indices cho từng frame
    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + \
              np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    
    # Đảm bảo indices không vượt quá độ dài tín hiệu
    indices = np.minimum(indices, len(padded_signal) - 1)
    
    # Trích xuất các frame từ tín hiệu
    frames = padded_signal[indices.astype(np.int32, copy=False)]
    
    return frames

def windowing(frames, window_type='bartlett'):
    """
    Áp dụng cửa sổ cho mỗi frame.
    
    Args:
        frames: Mảng 2D chứa các frame
        window_type: Loại cửa sổ ('hamming', 'hanning', 'blackman', 'bartlett')
        
    Returns:
        Các frame sau khi áp dụng cửa sổ
    """
    frame_length = frames.shape[1]
    
    if window_type == 'hamming':
        window = np.hamming(frame_length)
    elif window_type == 'hanning':
        window = np.hanning(frame_length)
    elif window_type == 'blackman':
        window = np.blackman(frame_length)
    elif window_type == 'bartlett':
        window = np.bartlett(frame_length)
    else:
        window = np.hamming(frame_length)  # Mặc định là hamming
        
    return frames * window

def fft_frames(frames, NFFT=512):
    """
    Tính FFT cho mỗi frame và lấy giá trị magnitude.
    
    Args:
        frames: Mảng 2D chứa các frame
        NFFT: Kích thước FFT (số điểm FFT)
        
    Returns:
        Magnitude của FFT
    """
    return np.absolute(np.fft.rfft(frames, NFFT))

def compute_fft_features(signal_in, sample_rate, frame_size=0.025, hop_length=256, 
                         overlap=0.6, NFFT=512, apply_log=True, window_type='bartlett',
                         normalize=False):
    """
    Tính toán đặc trưng FFT cho tín hiệu âm thanh:
      - Pre-emphasis, Framing, Windowing.
      - Tính FFT cho từng frame và lấy giá trị magnitude.
      - Trung bình các frame để có vector đặc trưng ổn định.
      
    Args:
        signal_in: Tín hiệu đầu vào
        sample_rate: Tần số lấy mẫu
        frame_size: Kích thước frame (giây hoặc số mẫu nếu > 1)
        hop_length: Bước nhảy (giây hoặc số mẫu nếu > 1). Nếu None, sẽ sử dụng overlap
        overlap: Tỷ lệ chồng lấp giữa các frame (0-1)
        NFFT: Số điểm FFT
        apply_log: Áp dụng logarit cho đặc trưng
        window_type: Loại cửa sổ ('hamming', 'hanning', 'blackman', 'bartlett')
        normalize: Chuẩn hóa đặc trưng
        
    Returns:
        Vector đặc trưng có kích thước (NFFT//2+1,)
    """
    # Áp dụng pre-emphasis
    emphasized_signal = pre_emphasis(signal_in)
    
    # Chia tín hiệu thành các frame
    frames = framing(emphasized_signal, sample_rate, frame_size, hop_length, overlap)
    
    # Áp dụng cửa sổ
    windowed_frames = windowing(frames, window_type)
    
    # Tính FFT
    mag_frames = fft_frames(windowed_frames, NFFT)
    
    # Trung bình theo các frame
    fft_feature = np.mean(mag_frames, axis=0)
    
    # Áp dụng logarithm (nếu cần)
    if apply_log:
        fft_feature = np.log(fft_feature + 1e-8)  # Thêm epsilon để tránh log(0)
    
    # Chuẩn hóa (nếu cần)
    if normalize:
        fft_feature = (fft_feature - np.mean(fft_feature)) / (np.std(fft_feature) + 1e-8)
    
    return fft_feature

def load_fft_features_from_directory(directory, sample_rate=22050, NFFT=512, 
                                     frame_size=0.025, hop_length=256, overlap=0.6,
                                     window_type='bartlett', output_dir=None, 
                                     dataset_type='train', normalize=False):
    """
    Duyệt qua các file âm thanh trong thư mục và tính đặc trưng FFT cho mỗi file.
    
    Args:
        directory: Thư mục chứa dữ liệu âm thanh
        sample_rate: Tần số lấy mẫu
        NFFT: Số điểm FFT
        frame_size: Kích thước frame (giây hoặc số mẫu nếu > 1)
        hop_length: Bước nhảy (giây hoặc số mẫu nếu > 1). Nếu None, sẽ sử dụng overlap
        overlap: Tỷ lệ chồng lấp giữa các frame (0-1)
        window_type: Loại cửa sổ ('hamming', 'hanning', 'blackman', 'bartlett')
        output_dir: Thư mục đầu ra để lưu đặc trưng
        dataset_type: Loại tập dữ liệu ('train', 'test', 'val')
        normalize: Chuẩn hóa đặc trưng
        
    Returns:
        features: Mảng đặc trưng
        labels: Nhãn tương ứng
    """
    # Kiểm tra nếu file đã tồn tại, thì load lại
    hop_str = f"{hop_length}" if hop_length else f"overlap{overlap}"
    features_file = os.path.join(output_dir, f'fft_features_{dataset_type}_bartlett_nfft{NFFT}_hop{hop_str}.pkl')
    labels_file = os.path.join(output_dir, f'fft_labels_{dataset_type}_bartlett_nfft{NFFT}_hop{hop_str}.pkl')
    
    if os.path.exists(features_file) and os.path.exists(labels_file):
        print(f"Loading {dataset_type} data from .pkl files...")
        features = joblib.load(features_file)
        labels = joblib.load(labels_file)
    else:
        print(f"Extracting {dataset_type} data...")
        start_time = time.time()
        labels = []
        features = []
        
        for label in ['bee', 'nobee', 'noqueen']:
            path = os.path.join(directory, label)
            if not os.path.exists(path):
                print(f"Warning: Path {path} does not exist. Skipping.")
                continue
                
            for file in os.listdir(path):
                if not file.endswith(('.wav', '.mp3', '.flac', '.ogg')):
                    continue
                    
                file_path = os.path.join(path, file)
                try:
                    signal, sr = librosa.load(file_path, sr=sample_rate)
                    fft_feature = compute_fft_features(
                        signal, sr, frame_size=frame_size, hop_length=hop_length, 
                        overlap=overlap, NFFT=NFFT, window_type=window_type,
                        normalize=normalize
                    )
                    features.append(fft_feature)
                    labels.append(label)
                except Exception as e:
                    print(f"Error processing file {file_path}: {str(e)}")
        
        features = np.array(features)
        labels = np.array(labels)
        
        # Lưu dữ liệu vào file .pkl nếu chưa tồn tại
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            joblib.dump(features, features_file)
            joblib.dump(labels, labels_file)
        
        end_time = time.time()
        print(f"FFT extraction time: {end_time - start_time:.2f} seconds")
        print(f"Features shape: {features.shape}, Labels shape: {labels.shape}")
    
    return features, labels

In [4]:
train_features_fft, train_labels_fft = load_fft_features_from_directory(train_path, sample_rate=22050, NFFT=2048, output_dir=output_dir, dataset_type='train')
val_features_fft, val_labels_fft = load_fft_features_from_directory(val_path, sample_rate=22050, NFFT=2048, output_dir=output_dir, dataset_type='val')
test_features_fft, test_labels_fft = load_fft_features_from_directory(test_path, sample_rate=22050, NFFT=2048, output_dir=output_dir, dataset_type='test')

Loading train data from .pkl files...
Loading val data from .pkl files...
Loading test data from .pkl files...


In [5]:
scaler = StandardScaler()
train_features_fft_scaled = scaler.fit_transform(train_features_fft)
val_features_fft_scaled = scaler.transform(val_features_fft)
test_features_fft_scaled = scaler.transform(test_features_fft)

In [19]:
start_time = time.time()

knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(train_features_fft, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_knn = knn_classifier.predict(val_features_fft)
test_accuracy_fft_knn = knn_classifier.predict(test_features_fft)
print(f"KNN (FFT features) - Validation Accuracy: {accuracy_score(val_labels_fft, val_pred_knn)*100:.2f}%")
print(f"KNN (FFT features) - Test Accuracy: {accuracy_score(test_labels_fft, test_accuracy_fft_knn)*100:.2f}%")

Training time: 0.03 seconds
KNN (FFT features) - Validation Accuracy: 86.15%
KNN (FFT features) - Test Accuracy: 86.16%


86.09%

86.20% (NFFT=2048, hop_length=256, window_type='hanning')

In [20]:
start_time = time.time()

knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(train_features_fft_scaled, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_knn = knn_classifier.predict(val_features_fft_scaled)
test_accuracy_fft_knn = knn_classifier.predict(test_features_fft_scaled)
print(f"KNN (FFT features) - Validation Accuracy: {accuracy_score(val_labels_fft, val_pred_knn)*100:.2f}%")
print(f"KNN (FFT features) - Test Accuracy: {accuracy_score(test_labels_fft, test_accuracy_fft_knn)*100:.2f}%")

Training time: 0.02 seconds
KNN (FFT features) - Validation Accuracy: 84.05%
KNN (FFT features) - Test Accuracy: 84.24%


84.86%

84.13% (NFFT=2048, hop_length=256, window_type='hanning')

In [ ]:
start_time = time.time()

svm_rbf_classifier = SVC(C=71.19418600172986, kernel='rbf', degree=3, gamma=0.03752055855124281, 
                         coef0=0.0, shrinking=True, probability=False, tol=0.0005319450186421158, 
                         cache_size=500, class_weight=None, verbose=False, max_iter=-1, 
                         decision_function_shape='ovr', break_ties=False, random_state=42)

# Kết hợp dữ liệu train và validation để tăng lượng dữ liệu huấn luyện (coi như là 8:2)
X_combined = np.vstack((train_features_fft, val_features_fft))
y_combined = np.concatenate((train_labels_fft, val_labels_fft))

# Huấn luyện mô hình SVM trên dữ liệu kết hợp
svm_rbf_classifier.fit(X_combined, y_combined)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ testing
test_predictions_svm_rbf = svm_rbf_classifier.predict(test_features_fft)
test_accuracy_fft_svm = accuracy_score(test_labels_fft, test_predictions_svm_rbf)
print(f"Test Accuracy (SVM with RBF Kernel): {test_accuracy_fft_svm * 100:.2f}%")

Training time: 46.71 seconds
Validation Accuracy (SVM with RBF Kernel): 100.00%
Test Accuracy (SVM with RBF Kernel): 88.22%


86.70%

87.86% (NFFT=2048, hop_length=256, window_type='hanning')

88.15% (NFFT=2048, hop_length=256, window_type='bartlett')


In [9]:
start_time = time.time()

svm_rbf_classifier = SVC(C=71.19418600172986, kernel='rbf', degree=3, gamma=0.03752055855124281, 
                         coef0=0.0, shrinking=True, probability=False, tol=0.0005319450186421158, 
                         cache_size=500, class_weight=None, verbose=False, max_iter=-1, 
                         decision_function_shape='ovr', break_ties=False, random_state=42)

# Huấn luyện mô hình SVM
svm_rbf_classifier.fit(train_features_fft_scaled, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ validation
val_predictions_svm_rbf = svm_rbf_classifier.predict(val_features_fft_scaled)
val_accuracy_svm_rbf = accuracy_score(val_labels_fft, val_predictions_svm_rbf)
print(f"Validation Accuracy (SVM with RBF Kernel): {val_accuracy_svm_rbf * 100:.2f}%")

# Đánh giá mô hình trên bộ testing
test_predictions_svm_rbf = svm_rbf_classifier.predict(test_features_fft_scaled)
scale_test_accuracy_fft_svm = accuracy_score(test_labels_fft, test_predictions_svm_rbf)
print(f"Test Accuracy (SVM with RBF Kernel): {scale_test_accuracy_fft_svm * 100:.2f}%")

Training time: 56.93 seconds
Validation Accuracy (SVM with RBF Kernel): 86.51%
Test Accuracy (SVM with RBF Kernel): 87.61%


86.99%

86.12% (NFFT=2048, hop_length=256, window_type='hanning')

86.05% (NFFT=2048, hop_length=256, window_type='bartlett')


In [ ]:
start_time = time.time()

lr_classifier = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=10, fit_intercept=True, intercept_scaling=1, 
                                   class_weight=None, random_state=42, solver='lbfgs', max_iter=1500, multi_class='deprecated', 
                                   verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
lr_classifier.fit(train_features_fft, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_lr = lr_classifier.predict(val_features_fft)
test_accuracy_fft_lr = lr_classifier.predict(test_features_fft)
print(f"Logistic Regression (FFT features) - Validation Accuracy: {accuracy_score(val_labels_fft, val_pred_lr)*100:.2f}%")
print(f"Logistic Regression (FFT features) - Test Accuracy: {accuracy_score(test_labels_fft, test_accuracy_fft_lr)*100:.2f}%")


Training time: 11.68 seconds
Logistic Regression (FFT features) - Validation Accuracy: 78.61%
Logistic Regression (FFT features) - Test Accuracy: 79.93%


C:\Users\dinhg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


79.89%

79.86% (NFFT=2048, hop_length=256, window_type='hanning')

80.04% (NFFT=2048, hop_length=256, window_type='bartlett')


In [ ]:
start_time = time.time()

lr_classifier = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=10, fit_intercept=True, intercept_scaling=1, 
                                   class_weight=None, random_state=42, solver='saga', max_iter=1000, multi_class='deprecated', 
                                   verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

lr_classifier.fit(train_features_fft_scaled, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_lr = lr_classifier.predict(val_features_fft_scaled)
scale_test_accuracy_fft_lr = lr_classifier.predict(test_features_fft_scaled)
print(f"Logistic Regression (FFT features) - Validation Accuracy: {accuracy_score(val_labels_fft, val_pred_lr)*100:.2f}%")
print(f"Logistic Regression (FFT features) - Test Accuracy: {accuracy_score(test_labels_fft, scale_test_accuracy_fft_lr)*100:.2f}%")


Training time: 145.44 seconds
Logistic Regression (FFT features) - Validation Accuracy: 78.75%
Logistic Regression (FFT features) - Test Accuracy: 80.11%


C:\Users\dinhg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


79.38%

80.14% (NFFT=2048, hop_length=256, window_type='hanning')

In [ ]:
start_time = time.time()

rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, 
                                       min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                       max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, 
                                       n_jobs=None, random_state=42, verbose=0, warm_start=False, class_weight=None, 
                                       ccp_alpha=0.0, max_samples=None, monotonic_cst=None)

rf_classifier.fit(train_features_fft, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_rf = rf_classifier.predict(val_features_fft)
test_accuracy_fft_rf = rf_classifier.predict(test_features_fft)
print(f"Random Forest (FFT features) - Validation Accuracy: {accuracy_score(val_labels_fft, val_pred_rf)*100:.2f}%")
print(f"Random Forest (FFT features) - Test Accuracy: {accuracy_score(test_labels_fft, test_accuracy_fft_rf)*100:.2f}%")

Training time: 34.56 seconds
Random Forest (FFT features) - Validation Accuracy: 84.70%
Random Forest (FFT features) - Test Accuracy: 84.28%


84.53%

84.78% (NFFT=2048, hop_length=256, window_type='hanning')

85.58% (NFFT=2048, hop_length=256, window_type='hamming')

In [10]:
start_time = time.time()

et_classifier = ExtraTreesClassifier(n_estimators=200, criterion='gini', max_depth=30, min_samples_split=2, min_samples_leaf=1, 
                                     min_weight_fraction_leaf=0.0, max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0,
                                     bootstrap=False, oob_score=False, n_jobs=None, random_state=42, verbose=0, warm_start=False, 
                                     class_weight=None, ccp_alpha=0.0, max_samples=None, monotonic_cst=None)

# Huấn luyện mô hình Extra Trees với dữ liệu đã chuẩn hóa
et_classifier.fit(train_features_fft, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ validation
val_predictions_et = et_classifier.predict(val_features_fft)
val_accuracy_et = accuracy_score(val_labels_fft, val_predictions_et)
print(f"Validation Accuracy (Extra Trees): {val_accuracy_et * 100:.2f}%")

# Đánh giá mô hình trên bộ testing
test_predictions_et = et_classifier.predict(test_features_fft)
test_accuracy_fft_et = accuracy_score(test_labels_fft, test_predictions_et)
print(f"Test Accuracy (Extra Trees): {test_accuracy_fft_et * 100:.2f}%")


Training time: 384.52 seconds
Validation Accuracy (Extra Trees): 87.02%
Test Accuracy (Extra Trees): 87.25%


86.81%

87.46% (NFFT=2048, hop_length=256, window_type='hanning')

In [ ]:
start_time = time.time()

et_classifier = ExtraTreesClassifier(n_estimators=200, criterion='gini', max_depth=30, min_samples_split=2, min_samples_leaf=1, 
                                     min_weight_fraction_leaf=0.0, max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0,
                                     bootstrap=False, oob_score=False, n_jobs=None, random_state=42, verbose=0, warm_start=False, 
                                     class_weight=None, ccp_alpha=0.0, max_samples=None, monotonic_cst=None)

# Huấn luyện mô hình Extra Trees với dữ liệu đã chuẩn hóa
et_classifier.fit(train_features_fft_scaled, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ validation
val_predictions_et = et_classifier.predict(val_features_fft_scaled)
val_accuracy_et = accuracy_score(val_labels_fft, val_predictions_et)
print(f"Validation Accuracy (Extra Trees): {val_accuracy_et * 100:.2f}%")

# Đánh giá mô hình trên bộ testing
test_predictions_et = et_classifier.predict(test_features_fft_scaled)
scale_test_accuracy_fft_et = accuracy_score(test_labels_fft, test_predictions_et)
print(f"Test Accuracy (Extra Trees): {scale_test_accuracy_fft_et * 100:.2f}%")


Training time: 404.47 seconds
Validation Accuracy (Extra Trees): 86.80%
Test Accuracy (Extra Trees): 86.45%


86.81%

87.28% (NFFT=2048, hop_length=256, window_type='hanning')

# FFTs + MFCC

In [48]:
def compute_mfcc_features(signal_in, sample_rate, n_mfcc=70, n_fft=2048, 
                           hop_length=512, n_mels=128, fmax=None):
    """
    Tính toán đặc trưng MFCC cho tín hiệu âm thanh.
    
    Args:
        signal_in: Tín hiệu đầu vào
        sample_rate: Tần số lấy mẫu
        n_mfcc: Số lượng hệ số MFCC
        n_fft: Kích thước FFT
        hop_length: Bước nhảy
        n_mels: Số lượng mel bands
        fmax: Tần số cao nhất
        
    Returns:
        Vector đặc trưng MFCC
    """
    # Tính MFCC
    mfccs = librosa.feature.mfcc(
        y=signal_in, 
        sr=sample_rate, 
        n_mfcc=n_mfcc,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels,
        fmax=fmax
    )
    
    # Trung bình theo thời gian để có vector đặc trưng ổn định
    mfcc_feature = np.mean(mfccs, axis=1)
    
    return mfcc_feature

def load_combined_features_from_directory(directory, sample_rate=22050, 
                                          NFFT=512, n_mfcc=70, 
                                          frame_size=0.025, hop_length=256, 
                                          overlap=0.6, output_dir=None, 
                                          dataset_type='train', normalize=False):
    """
    Duyệt qua các file âm thanh và tính đặc trưng kết hợp FFT và MFCC.
    
    Args:
        directory: Thư mục chứa dữ liệu âm thanh
        sample_rate: Tần số lấy mẫu
        NFFT: Số điểm FFT
        n_mfcc: Số lượng hệ số MFCC
        frame_size: Kích thước frame
        hop_length: Bước nhảy
        overlap: Tỷ lệ chồng lấp giữa các frame
        output_dir: Thư mục đầu ra để lưu đặc trưng
        dataset_type: Loại tập dữ liệu ('train', 'test', 'val')
        normalize: Chuẩn hóa đặc trưng
        
    Returns:
        features: Mảng đặc trưng kết hợp
        labels: Nhãn tương ứng
    """
    # Kiểm tra nếu file đã tồn tại, thì load lại
    hop_str = f"{hop_length}" if hop_length else f"overlap{overlap}"
    features_file = os.path.join(output_dir, f'combined_features_{dataset_type}_nfft{NFFT}_hop{hop_str}.pkl')
    labels_file = os.path.join(output_dir, f'combined_labels_{dataset_type}_nfft{NFFT}_hop{hop_str}.pkl')
    
    if os.path.exists(features_file) and os.path.exists(labels_file):
        print(f"Loading {dataset_type} data from .pkl files...")
        features = joblib.load(features_file)
        labels = joblib.load(labels_file)
    else:
        print(f"Extracting {dataset_type} data...")
        start_time = time.time()
        labels = []
        features = []
        
        for label in ['bee', 'nobee', 'noqueen']:
            path = os.path.join(directory, label)
            if not os.path.exists(path):
                print(f"Warning: Path {path} does not exist. Skipping.")
                continue
                
            for file in os.listdir(path):
                if not file.endswith(('.wav', '.mp3', '.flac', '.ogg')):
                    continue
                    
                file_path = os.path.join(path, file)
                try:
                    # Tải tín hiệu âm thanh
                    signal, sr = librosa.load(file_path, sr=sample_rate)
                    
                    # Tính FFT features từ code cũ
                    fft_feature = compute_fft_features(
                        signal, sr, frame_size=frame_size, hop_length=hop_length, 
                        overlap=overlap, NFFT=NFFT, window_type='bartlett',
                        normalize=normalize
                    )
                    
                    # Tính MFCC features
                    mfcc_feature = compute_mfcc_features(
                        signal, sr, n_mfcc=n_mfcc, n_fft=NFFT, 
                        hop_length=hop_length
                    )
                    
                    # Kết hợp đặc trưng FFT và MFCC
                    combined_feature = np.concatenate([fft_feature, mfcc_feature])
                    
                    features.append(combined_feature)
                    labels.append(label)
                except Exception as e:
                    print(f"Error processing file {file_path}: {str(e)}")
        
        features = np.array(features)
        labels = np.array(labels)
        
        # Lưu dữ liệu vào file .pkl nếu chưa tồn tại
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            joblib.dump(features, features_file)
            joblib.dump(labels, labels_file)
        
        end_time = time.time()
        print(f"Combined features extraction time: {end_time - start_time:.2f} seconds")
        print(f"Features shape: {features.shape}, Labels shape: {labels.shape}")
    
    return features, labels

def pre_emphasis(signal_in, pre_emph=0.97):
    """
    Áp dụng pre-emphasis để nhấn mạnh các tần số cao.
    
    Args:
        signal_in: Tín hiệu đầu vào
        pre_emph: Hệ số pre-emphasis, mặc định là 0.97
        
    Returns:
        Tín hiệu sau khi áp dụng pre-emphasis
    """
    return np.append(signal_in[0], signal_in[1:] - pre_emph * signal_in[:-1])

def framing(signal_in, sample_rate, frame_size=0.025, hop_length=256, overlap=0.6):
    """
    Chia tín hiệu thành các frame với kích thước và bước nhảy xác định.
    
    Args:
        signal_in: Tín hiệu đầu vào
        sample_rate: Tần số lấy mẫu
        frame_size: Kích thước frame (giây hoặc số mẫu nếu frame_size > 1)
        hop_length: Bước nhảy (giây hoặc số mẫu nếu hop_length > 1)
                    Nếu None, sẽ được tính dựa trên overlap
        overlap: Tỷ lệ chồng lấp giữa các frame, mặc định là 0.6 (60%)
                Chỉ được sử dụng khi hop_length là None
        
    Returns:
        Mảng 2D chứa các frame
    """
    # Xác định kích thước frame trong số mẫu
    if frame_size <= 1:  # Nếu frame_size <= 1, xem như đơn vị là giây
        frame_length = int(round(frame_size * sample_rate))
    else:  # Ngược lại, xem như đơn vị là số mẫu
        frame_length = int(frame_size)
    
    # Xác định hop_length (bước nhảy) trong số mẫu
    if hop_length is None:
        # Tính hop_length từ overlap
        frame_step = int(frame_length * (1 - overlap))
    elif hop_length <= 1:  # Nếu hop_length <= 1, xem như đơn vị là giây
        frame_step = int(round(hop_length * sample_rate))
    else:  # Ngược lại, xem như đơn vị là số mẫu
        frame_step = int(hop_length)
    
    # Đảm bảo hop_length không quá nhỏ
    frame_step = max(1, frame_step)
    
    signal_length = len(signal_in)
    
    # Tính số frame cần thiết
    num_frames = int(np.ceil((signal_length - frame_length) / frame_step)) + 1
    
    # Đệm tín hiệu để đảm bảo có đủ mẫu cho tất cả các frame
    pad_signal_length = (num_frames - 1) * frame_step + frame_length
    pad_zeros = np.zeros(pad_signal_length - signal_length)
    padded_signal = np.concatenate([signal_in, pad_zeros])
    
    # Tạo indices cho từng frame
    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + \
              np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    
    # Đảm bảo indices không vượt quá độ dài tín hiệu
    indices = np.minimum(indices, len(padded_signal) - 1)
    
    # Trích xuất các frame từ tín hiệu
    frames = padded_signal[indices.astype(np.int32, copy=False)]
    
    return frames

def windowing(frames, window_type='bartlett'):
    """
    Áp dụng cửa sổ cho mỗi frame.
    
    Args:
        frames: Mảng 2D chứa các frame
        window_type: Loại cửa sổ ('hamming', 'hanning', 'blackman', 'bartlett')
        
    Returns:
        Các frame sau khi áp dụng cửa sổ
    """
    frame_length = frames.shape[1]
    
    if window_type == 'hamming':
        window = np.hamming(frame_length)
    elif window_type == 'hanning':
        window = np.hanning(frame_length)
    elif window_type == 'blackman':
        window = np.blackman(frame_length)
    elif window_type == 'bartlett':
        window = np.bartlett(frame_length)
    else:
        window = np.hamming(frame_length)  # Mặc định là hamming
        
    return frames * window

def fft_frames(frames, NFFT=512):
    """
    Tính FFT cho mỗi frame và lấy giá trị magnitude.
    
    Args:
        frames: Mảng 2D chứa các frame
        NFFT: Kích thước FFT (số điểm FFT)
        
    Returns:
        Magnitude của FFT
    """
    return np.absolute(np.fft.rfft(frames, NFFT))

def compute_fft_features(signal_in, sample_rate, frame_size=0.025, hop_length=256, 
                         overlap=0.6, NFFT=512, apply_log=True, window_type='bartlett',
                         normalize=False):
    """
    Tính toán đặc trưng FFT cho tín hiệu âm thanh:
      - Pre-emphasis, Framing, Windowing.
      - Tính FFT cho từng frame và lấy giá trị magnitude.
      - Trung bình các frame để có vector đặc trưng ổn định.
      
    Args:
        signal_in: Tín hiệu đầu vào
        sample_rate: Tần số lấy mẫu
        frame_size: Kích thước frame (giây hoặc số mẫu nếu > 1)
        hop_length: Bước nhảy (giây hoặc số mẫu nếu > 1). Nếu None, sẽ sử dụng overlap
        overlap: Tỷ lệ chồng lấp giữa các frame (0-1)
        NFFT: Số điểm FFT
        apply_log: Áp dụng logarit cho đặc trưng
        window_type: Loại cửa sổ ('hamming', 'hanning', 'blackman', 'bartlett')
        normalize: Chuẩn hóa đặc trưng
        
    Returns:
        Vector đặc trưng có kích thước (NFFT//2+1,)
    """
    # Áp dụng pre-emphasis
    emphasized_signal = pre_emphasis(signal_in)
    
    # Chia tín hiệu thành các frame
    frames = framing(emphasized_signal, sample_rate, frame_size, hop_length, overlap)
    
    # Áp dụng cửa sổ
    windowed_frames = windowing(frames, window_type)
    
    # Tính FFT
    mag_frames = fft_frames(windowed_frames, NFFT)
    
    # Trung bình theo các frame
    fft_feature = np.mean(mag_frames, axis=0)
    
    # Áp dụng logarithm (nếu cần)
    if apply_log:
        fft_feature = np.log(fft_feature + 1e-8)  # Thêm epsilon để tránh log(0)
    
    # Chuẩn hóa (nếu cần)
    if normalize:
        fft_feature = (fft_feature - np.mean(fft_feature)) / (np.std(fft_feature) + 1e-8)
    
    return fft_feature

In [49]:
train_features_fft, train_labels_fft = load_combined_features_from_directory(train_path, sample_rate=22050, NFFT=2048, output_dir=output_dir, dataset_type='train')
val_features_fft, val_labels_fft = load_combined_features_from_directory(val_path, sample_rate=22050, NFFT=2048, output_dir=output_dir, dataset_type='val')
test_features_fft, test_labels_fft = load_combined_features_from_directory(test_path, sample_rate=22050, NFFT=2048, output_dir=output_dir, dataset_type='test')

Loading train data from .pkl files...
Loading val data from .pkl files...
Loading test data from .pkl files...


In [21]:
scaler = StandardScaler()
train_features_fft_scaled = scaler.fit_transform(train_features_fft)
val_features_fft_scaled = scaler.transform(val_features_fft)
test_features_fft_scaled = scaler.transform(test_features_fft)

In [22]:
start_time = time.time()

knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(train_features_fft, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_knn = knn_classifier.predict(val_features_fft)
test_accuracy_fft_knn = knn_classifier.predict(test_features_fft)
print(f"KNN (FFT features) - Validation Accuracy: {accuracy_score(val_labels_fft, val_pred_knn)*100:.2f}%")
print(f"KNN (FFT features) - Test Accuracy: {accuracy_score(test_labels_fft, test_accuracy_fft_knn)*100:.2f}%")

Training time: 0.02 seconds
KNN (FFT features) - Validation Accuracy: 87.24%
KNN (FFT features) - Test Accuracy: 87.07%


In [23]:
start_time = time.time()

knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(train_features_fft_scaled, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_knn = knn_classifier.predict(val_features_fft_scaled)
test_accuracy_fft_knn = knn_classifier.predict(test_features_fft_scaled)
print(f"KNN (FFT features) - Validation Accuracy: {accuracy_score(val_labels_fft, val_pred_knn)*100:.2f}%")
print(f"KNN (FFT features) - Test Accuracy: {accuracy_score(test_labels_fft, test_accuracy_fft_knn)*100:.2f}%")

Training time: 0.02 seconds
KNN (FFT features) - Validation Accuracy: 86.66%
KNN (FFT features) - Test Accuracy: 86.85%


In [51]:
start_time = time.time()

svm_rbf_classifier = SVC(C=100, kernel='rbf', degree=3, gamma=0.1, coef0=0.0, shrinking=True, 
                         probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, 
                         max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=42)

# Huấn luyện mô hình SVM
svm_rbf_classifier.fit(train_features_fft, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ validation
val_predictions_svm_rbf = svm_rbf_classifier.predict(val_features_fft)
val_accuracy_svm_rbf = accuracy_score(val_labels_fft, val_predictions_svm_rbf)
print(f"Validation Accuracy (SVM with RBF Kernel): {val_accuracy_svm_rbf * 100:.2f}%")

# Đánh giá mô hình trên bộ testing
test_predictions_svm_rbf = svm_rbf_classifier.predict(test_features_fft)
test_accuracy_fft_svm = accuracy_score(test_labels_fft, test_predictions_svm_rbf)
print(f"Test Accuracy (SVM with RBF Kernel): {test_accuracy_fft_svm * 100:.2f}%")

Training time: 123.98 seconds
Validation Accuracy (SVM with RBF Kernel): 40.25%
Test Accuracy (SVM with RBF Kernel): 40.11%


In [25]:
start_time = time.time()

svm_rbf_classifier = SVC(C=71.19418600172986, kernel='rbf', degree=3, gamma=0.03752055855124281, 
                         coef0=0.0, shrinking=True, probability=False, tol=0.0005319450186421158, 
                         cache_size=500, class_weight=None, verbose=False, max_iter=-1, 
                         decision_function_shape='ovr', break_ties=False, random_state=42)

# Huấn luyện mô hình SVM
svm_rbf_classifier.fit(train_features_fft, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ validation
val_predictions_svm_rbf = svm_rbf_classifier.predict(val_features_fft)
val_accuracy_svm_rbf = accuracy_score(val_labels_fft, val_predictions_svm_rbf)
print(f"Validation Accuracy (SVM with RBF Kernel): {val_accuracy_svm_rbf * 100:.2f}%")

# Đánh giá mô hình trên bộ testing
test_predictions_svm_rbf = svm_rbf_classifier.predict(test_features_fft)
test_accuracy_fft_svm = accuracy_score(test_labels_fft, test_predictions_svm_rbf)
print(f"Test Accuracy (SVM with RBF Kernel): {test_accuracy_fft_svm * 100:.2f}%")

Training time: 115.63 seconds
Validation Accuracy (SVM with RBF Kernel): 76.21%
Test Accuracy (SVM with RBF Kernel): 73.08%


In [26]:
start_time = time.time()

svm_rbf_classifier = SVC(C=71.19418600172986, kernel='rbf', degree=3, gamma=0.03752055855124281, 
                         coef0=0.0, shrinking=True, probability=False, tol=0.0005319450186421158, 
                         cache_size=500, class_weight=None, verbose=False, max_iter=-1, 
                         decision_function_shape='ovr', break_ties=False, random_state=42)

# Kết hợp dữ liệu train và validation để tăng lượng dữ liệu huấn luyện (coi như là 8:2)
X_combined = np.vstack((train_features_fft, val_features_fft))
y_combined = np.concatenate((train_labels_fft, val_labels_fft))

# Huấn luyện mô hình SVM trên dữ liệu kết hợp
svm_rbf_classifier.fit(X_combined, y_combined)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ testing
test_predictions_svm_rbf = svm_rbf_classifier.predict(test_features_fft)
test_accuracy_fft_svm = accuracy_score(test_labels_fft, test_predictions_svm_rbf)
print(f"Test Accuracy (SVM with RBF Kernel): {test_accuracy_fft_svm * 100:.2f}%")

Training time: 155.32 seconds
Test Accuracy (SVM with RBF Kernel): 73.80%


In [27]:
start_time = time.time()

lr_classifier = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=10, fit_intercept=True, intercept_scaling=1, 
                                   class_weight=None, random_state=42, solver='lbfgs', max_iter=1500, multi_class='deprecated', 
                                   verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
lr_classifier.fit(train_features_fft, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_lr = lr_classifier.predict(val_features_fft)
test_accuracy_fft_lr = lr_classifier.predict(test_features_fft)
print(f"Logistic Regression (FFT features) - Validation Accuracy: {accuracy_score(val_labels_fft, val_pred_lr)*100:.2f}%")
print(f"Logistic Regression (FFT features) - Test Accuracy: {accuracy_score(test_labels_fft, test_accuracy_fft_lr)*100:.2f}%")


Training time: 19.83 seconds
Logistic Regression (FFT features) - Validation Accuracy: 80.42%
Logistic Regression (FFT features) - Test Accuracy: 80.00%


C:\Users\dinhg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [28]:
start_time = time.time()

lr_classifier = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=10, fit_intercept=True, intercept_scaling=1, 
                                   class_weight=None, random_state=42, solver='saga', max_iter=1000, multi_class='deprecated', 
                                   verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

lr_classifier.fit(train_features_fft_scaled, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_lr = lr_classifier.predict(val_features_fft_scaled)
scale_test_accuracy_fft_lr = lr_classifier.predict(test_features_fft_scaled)
print(f"Logistic Regression (FFT features) - Validation Accuracy: {accuracy_score(val_labels_fft, val_pred_lr)*100:.2f}%")
print(f"Logistic Regression (FFT features) - Test Accuracy: {accuracy_score(test_labels_fft, scale_test_accuracy_fft_lr)*100:.2f}%")


Training time: 302.90 seconds
Logistic Regression (FFT features) - Validation Accuracy: 79.77%
Logistic Regression (FFT features) - Test Accuracy: 80.87%


C:\Users\dinhg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [29]:
start_time = time.time()

rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, 
                                       min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                       max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, 
                                       n_jobs=None, random_state=42, verbose=0, warm_start=False, class_weight=None, 
                                       ccp_alpha=0.0, max_samples=None, monotonic_cst=None)

rf_classifier.fit(train_features_fft, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_rf = rf_classifier.predict(val_features_fft)
test_accuracy_fft_rf = rf_classifier.predict(test_features_fft)
print(f"Random Forest (FFT features) - Validation Accuracy: {accuracy_score(val_labels_fft, val_pred_rf)*100:.2f}%")
print(f"Random Forest (FFT features) - Test Accuracy: {accuracy_score(test_labels_fft, test_accuracy_fft_rf)*100:.2f}%")

Training time: 52.99 seconds
Random Forest (FFT features) - Validation Accuracy: 86.00%
Random Forest (FFT features) - Test Accuracy: 86.16%


In [30]:
start_time = time.time()

et_classifier = ExtraTreesClassifier(n_estimators=200, criterion='gini', max_depth=30, min_samples_split=2, min_samples_leaf=1, 
                                     min_weight_fraction_leaf=0.0, max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0,
                                     bootstrap=False, oob_score=False, n_jobs=None, random_state=42, verbose=0, warm_start=False, 
                                     class_weight=None, ccp_alpha=0.0, max_samples=None, monotonic_cst=None)

# Huấn luyện mô hình Extra Trees với dữ liệu đã chuẩn hóa
et_classifier.fit(train_features_fft, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ validation
val_predictions_et = et_classifier.predict(val_features_fft)
val_accuracy_et = accuracy_score(val_labels_fft, val_predictions_et)
print(f"Validation Accuracy (Extra Trees): {val_accuracy_et * 100:.2f}%")

# Đánh giá mô hình trên bộ testing
test_predictions_et = et_classifier.predict(test_features_fft)
test_accuracy_fft_et = accuracy_score(test_labels_fft, test_predictions_et)
print(f"Test Accuracy (Extra Trees): {test_accuracy_fft_et * 100:.2f}%")


Training time: 420.69 seconds
Validation Accuracy (Extra Trees): 86.80%
Test Accuracy (Extra Trees): 87.46%


In [31]:
start_time = time.time()

et_classifier = ExtraTreesClassifier(n_estimators=200, criterion='gini', max_depth=30, min_samples_split=2, min_samples_leaf=1, 
                                     min_weight_fraction_leaf=0.0, max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0,
                                     bootstrap=False, oob_score=False, n_jobs=None, random_state=42, verbose=0, warm_start=False, 
                                     class_weight=None, ccp_alpha=0.0, max_samples=None, monotonic_cst=None)

# Huấn luyện mô hình Extra Trees với dữ liệu đã chuẩn hóa
et_classifier.fit(train_features_fft_scaled, train_labels_fft)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ validation
val_predictions_et = et_classifier.predict(val_features_fft_scaled)
val_accuracy_et = accuracy_score(val_labels_fft, val_predictions_et)
print(f"Validation Accuracy (Extra Trees): {val_accuracy_et * 100:.2f}%")

# Đánh giá mô hình trên bộ testing
test_predictions_et = et_classifier.predict(test_features_fft_scaled)
scale_test_accuracy_fft_et = accuracy_score(test_labels_fft, test_predictions_et)
print(f"Test Accuracy (Extra Trees): {scale_test_accuracy_fft_et * 100:.2f}%")


Training time: 421.90 seconds
Validation Accuracy (Extra Trees): 86.95%
Test Accuracy (Extra Trees): 87.57%


In [30]:
def pre_emphasis(signal_in, pre_emph=0.97):
    """
    Áp dụng pre-emphasis để nhấn mạnh các tần số cao.
    
    Args:
        signal_in: Tín hiệu đầu vào
        pre_emph: Hệ số pre-emphasis, mặc định là 0.97
        
    Returns:
        Tín hiệu sau khi áp dụng pre-emphasis
    """
    return np.append(signal_in[0], signal_in[1:] - pre_emph * signal_in[:-1])

def framing(signal_in, sample_rate, frame_size=0.025, hop_length=256, overlap=0.6):
    """
    Chia tín hiệu thành các frame với kích thước và bước nhảy xác định.
    
    Args:
        signal_in: Tín hiệu đầu vào
        sample_rate: Tần số lấy mẫu
        frame_size: Kích thước frame (giây hoặc số mẫu nếu frame_size > 1)
        hop_length: Bước nhảy (giây hoặc số mẫu nếu hop_length > 1)
                    Nếu None, sẽ được tính dựa trên overlap
        overlap: Tỷ lệ chồng lấp giữa các frame, mặc định là 0.6 (60%)
                Chỉ được sử dụng khi hop_length là None
        
    Returns:
        Mảng 2D chứa các frame
    """
    # Xác định kích thước frame trong số mẫu
    if frame_size <= 1:  # Nếu frame_size <= 1, xem như đơn vị là giây
        frame_length = int(round(frame_size * sample_rate))
    else:  # Ngược lại, xem như đơn vị là số mẫu
        frame_length = int(frame_size)
    
    # Xác định hop_length (bước nhảy) trong số mẫu
    if hop_length is None:
        # Tính hop_length từ overlap
        frame_step = int(frame_length * (1 - overlap))
    elif hop_length <= 1:  # Nếu hop_length <= 1, xem như đơn vị là giây
        frame_step = int(round(hop_length * sample_rate))
    else:  # Ngược lại, xem như đơn vị là số mẫu
        frame_step = int(hop_length)
    
    # Đảm bảo hop_length không quá nhỏ
    frame_step = max(1, frame_step)
    
    signal_length = len(signal_in)
    
    # Tính số frame cần thiết
    num_frames = int(np.ceil((signal_length - frame_length) / frame_step)) + 1
    
    # Đệm tín hiệu để đảm bảo có đủ mẫu cho tất cả các frame
    pad_signal_length = (num_frames - 1) * frame_step + frame_length
    pad_zeros = np.zeros(pad_signal_length - signal_length)
    padded_signal = np.concatenate([signal_in, pad_zeros])
    
    # Tạo indices cho từng frame
    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + \
              np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    
    # Đảm bảo indices không vượt quá độ dài tín hiệu
    indices = np.minimum(indices, len(padded_signal) - 1)
    
    # Trích xuất các frame từ tín hiệu
    frames = padded_signal[indices.astype(np.int32, copy=False)]
    
    return frames

def windowing(frames, window_type='bartlett'):
    """
    Áp dụng cửa sổ cho mỗi frame.
    
    Args:
        frames: Mảng 2D chứa các frame
        window_type: Loại cửa sổ ('hamming', 'hanning', 'blackman', 'bartlett')
        
    Returns:
        Các frame sau khi áp dụng cửa sổ
    """
    frame_length = frames.shape[1]
    
    if window_type == 'hamming':
        window = np.hamming(frame_length)
    elif window_type == 'hanning':
        window = np.hanning(frame_length)
    elif window_type == 'blackman':
        window = np.blackman(frame_length)
    elif window_type == 'bartlett':
        window = np.bartlett(frame_length)
    else:
        window = np.hamming(frame_length)  # Mặc định là hamming
        
    return frames * window

def fft_frames(frames, NFFT=512):
    """
    Tính FFT cho mỗi frame và lấy giá trị magnitude.
    
    Args:
        frames: Mảng 2D chứa các frame
        NFFT: Kích thước FFT (số điểm FFT)
        
    Returns:
        Magnitude của FFT
    """
    return np.absolute(np.fft.rfft(frames, NFFT))

def compute_fft_features(signal_in, sample_rate, frame_size=0.025, hop_length=256, 
                         overlap=0.6, NFFT=512, apply_log=True, window_type='bartlett',
                         normalize=False):
    """
    Tính toán đặc trưng FFT cho tín hiệu âm thanh:
      - Pre-emphasis, Framing, Windowing.
      - Tính FFT cho từng frame và lấy giá trị magnitude.
      - Trung bình các frame để có vector đặc trưng ổn định.
      
    Args:
        signal_in: Tín hiệu đầu vào
        sample_rate: Tần số lấy mẫu
        frame_size: Kích thước frame (giây hoặc số mẫu nếu > 1)
        hop_length: Bước nhảy (giây hoặc số mẫu nếu > 1). Nếu None, sẽ sử dụng overlap
        overlap: Tỷ lệ chồng lấp giữa các frame (0-1)
        NFFT: Số điểm FFT
        apply_log: Áp dụng logarit cho đặc trưng
        window_type: Loại cửa sổ ('hamming', 'hanning', 'blackman', 'bartlett')
        normalize: Chuẩn hóa đặc trưng
        
    Returns:
        Vector đặc trưng có kích thước (NFFT//2+1,)
    """
    # Áp dụng pre-emphasis
    emphasized_signal = pre_emphasis(signal_in)
    
    # Chia tín hiệu thành các frame
    frames = framing(emphasized_signal, sample_rate, frame_size, hop_length, overlap)
    
    # Áp dụng cửa sổ
    windowed_frames = windowing(frames, window_type)
    
    # Tính FFT
    mag_frames = fft_frames(windowed_frames, NFFT)
    
    # Trung bình theo các frame
    fft_feature = np.mean(mag_frames, axis=0)
    
    # Áp dụng logarithm (nếu cần)
    if apply_log:
        fft_feature = np.log(fft_feature + 1e-8)  # Thêm epsilon để tránh log(0)
    
    # Chuẩn hóa (nếu cần)
    if normalize:
        fft_feature = (fft_feature - np.mean(fft_feature)) / (np.std(fft_feature) + 1e-8)
    
    return fft_feature

def compute_cqt_features(signal_in, sample_rate, hop_length=512, fmin=None, 
                         n_bins=84, bins_per_octave=12, apply_log=True, normalize=False):
    """
    Tính đặc trưng CQT của tín hiệu:
      - Sử dụng librosa.cqt để tính ma trận CQT, lấy giá trị magnitude.
      - (Tùy chọn) Áp dụng log scaling để giảm phạm vi giá trị.
      - Trung bình theo trục thời gian (axis=1) để thu được vector đặc trưng có kích thước (n_bins,).
      
    Args:
        signal_in: Tín hiệu đầu vào
        sample_rate: Tần số lấy mẫu
        hop_length: Bước nhảy giữa các cửa sổ phân tích liên tiếp
        fmin: Tần số thấp nhất (Hz). Nếu None, librosa sẽ sử dụng mặc định
        n_bins: Số lượng bin tần số
        bins_per_octave: Số lượng bin cho mỗi quãng 8
        apply_log: Áp dụng logarit cho đặc trưng
        normalize: Chuẩn hóa đặc trưng
        
    Returns:
        Vector đặc trưng CQT có kích thước (n_bins,)
    """
    cqt_matrix = np.abs(librosa.cqt(signal_in, sr=sample_rate, hop_length=hop_length,
                                    fmin=fmin, n_bins=n_bins, bins_per_octave=bins_per_octave))
    
    # Trung bình theo trục thời gian (tính trung bình các cột)
    cqt_feature = np.mean(cqt_matrix, axis=1)
    
    # Áp dụng logarithm (nếu cần)
    if apply_log:
        cqt_feature = np.log(cqt_feature + 1e-8)  # Thêm epsilon để tránh log(0)
    
    # Chuẩn hóa (nếu cần)
    if normalize:
        cqt_feature = (cqt_feature - np.mean(cqt_feature)) / (np.std(cqt_feature) + 1e-8)
    
    return cqt_feature

def compute_features(signal_in, sample_rate, frame_size=0.025, hop_length=512, 
                     overlap=0.6, NFFT=512, window_type='bartlett', normalize=False,
                     fmin=None, n_bins=84, bins_per_octave=12, apply_log=True):
    """
    Tính toán và kết hợp đặc trưng FFT và CQT cho tín hiệu âm thanh.
    
    Args:
        signal_in: Tín hiệu đầu vào
        sample_rate: Tần số lấy mẫu
        frame_size: Kích thước frame cho FFT (giây hoặc số mẫu nếu > 1)
        hop_length: Bước nhảy (giây hoặc số mẫu nếu > 1)
        overlap: Tỷ lệ chồng lấp giữa các frame (0-1) cho FFT
        NFFT: Số điểm FFT
        window_type: Loại cửa sổ ('hamming', 'hanning', 'blackman', 'bartlett')
        normalize: Chuẩn hóa đặc trưng
        fmin: Tần số thấp nhất (Hz) cho CQT
        n_bins: Số lượng bin tần số cho CQT
        bins_per_octave: Số lượng bin cho mỗi quãng 8 cho CQT
        apply_log: Áp dụng logarit cho đặc trưng
        
    Returns:
        Vector đặc trưng kết hợp FFT và CQT
    """
    # Tính đặc trưng FFT
    fft_feature = compute_fft_features(
        signal_in, sample_rate, frame_size=frame_size, hop_length=hop_length, 
        overlap=overlap, NFFT=NFFT, window_type=window_type,
        apply_log=apply_log, normalize=normalize
    )
    
    # Tính đặc trưng CQT
    cqt_feature = compute_cqt_features(
        signal_in, sample_rate, hop_length=hop_length, fmin=fmin,
        n_bins=n_bins, bins_per_octave=bins_per_octave,
        apply_log=apply_log, normalize=normalize
    )
    
    # Kết hợp hai đặc trưng
    combined_feature = np.concatenate((fft_feature, cqt_feature))
    
    return combined_feature

def load_features_from_directory(directory, sample_rate=22050, NFFT=512, 
                                 frame_size=0.025, hop_length=512, overlap=0.6,
                                 window_type='bartlett', output_dir=None, 
                                 dataset_type='train', normalize=False,
                                 fmin=None, n_bins=84, bins_per_octave=12):
    """
    Duyệt qua các file âm thanh trong thư mục và tính đặc trưng kết hợp FFT và CQT cho mỗi file.
    
    Args:
        directory: Thư mục chứa dữ liệu âm thanh
        sample_rate: Tần số lấy mẫu
        NFFT: Số điểm FFT
        frame_size: Kích thước frame (giây hoặc số mẫu nếu > 1)
        hop_length: Bước nhảy (giây hoặc số mẫu nếu > 1)
        overlap: Tỷ lệ chồng lấp giữa các frame (0-1)
        window_type: Loại cửa sổ ('hamming', 'hanning', 'blackman', 'bartlett')
        output_dir: Thư mục đầu ra để lưu đặc trưng
        dataset_type: Loại tập dữ liệu ('train', 'test', 'val')
        normalize: Chuẩn hóa đặc trưng
        fmin: Tần số thấp nhất (Hz) cho CQT
        n_bins: Số lượng bin tần số cho CQT
        bins_per_octave: Số lượng bin cho mỗi quãng 8 cho CQT
        
    Returns:
        features: Mảng đặc trưng kết hợp
        labels: Nhãn tương ứng
    """
    # Kiểm tra nếu file đã tồn tại, thì load lại
    hop_str = f"{hop_length}" if hop_length else f"overlap{overlap}"
    features_file = os.path.join(output_dir, f'combined_features_{dataset_type}_nfft{NFFT}_hop{hop_str}_bins{n_bins}.pkl')
    labels_file = os.path.join(output_dir, f'combined_labels_{dataset_type}_nfft{NFFT}_hop{hop_str}_bins{n_bins}.pkl')
    
    if os.path.exists(features_file) and os.path.exists(labels_file):
        print(f"Loading {dataset_type} data from .pkl files...")
        features = joblib.load(features_file)
        labels = joblib.load(labels_file)
    else:
        print(f"Extracting {dataset_type} data...")
        start_time = time.time()
        labels = []
        features = []
        
        for label in ['bee', 'nobee', 'noqueen']:
            path = os.path.join(directory, label)
            if not os.path.exists(path):
                print(f"Warning: Path {path} does not exist. Skipping.")
                continue
                
            for file in os.listdir(path):
                if not file.endswith(('.wav', '.mp3', '.flac', '.ogg')):
                    continue
                    
                file_path = os.path.join(path, file)
                try:
                    signal, sr = librosa.load(file_path, sr=sample_rate)
                    
                    # Tính toán đặc trưng kết hợp FFT và CQT
                    combined_feature = compute_features(
                        signal, sr, frame_size=frame_size, hop_length=hop_length, 
                        overlap=overlap, NFFT=NFFT, window_type=window_type,
                        normalize=normalize, fmin=fmin, n_bins=n_bins, 
                        bins_per_octave=bins_per_octave
                    )
                    
                    features.append(combined_feature)
                    labels.append(label)
                except Exception as e:
                    print(f"Error processing file {file_path}: {str(e)}")
        
        features = np.array(features)
        labels = np.array(labels)
        
        # Lưu dữ liệu vào file .pkl nếu chưa tồn tại
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            joblib.dump(features, features_file)
            joblib.dump(labels, labels_file)
        
        end_time = time.time()
        print(f"Feature extraction time: {end_time - start_time:.2f} seconds")
        print(f"Features shape: {features.shape}, Labels shape: {labels.shape}")
    
    return features, labels

# Trích xuất đặc trưng kết hợp cho tập dữ liệu
train_features, train_labels = load_features_from_directory(
    train_path, sample_rate=22050, NFFT=2048, 
    hop_length=512, fmin=None, n_bins=96, bins_per_octave=12,
    output_dir=output_dir, dataset_type='train'
)

val_features, val_labels = load_features_from_directory(
    val_path, sample_rate=22050, NFFT=2048, 
    hop_length=512, fmin=None, n_bins=96, bins_per_octave=12,
    output_dir=output_dir, dataset_type='val'
)

test_features, test_labels = load_features_from_directory(
    test_path, sample_rate=22050, NFFT=2048, 
    hop_length=512, fmin=None, n_bins=96, bins_per_octave=12,
    output_dir=output_dir, dataset_type='test'
)

Loading train data from .pkl files...
Loading val data from .pkl files...
Loading test data from .pkl files...


In [31]:
train_features.shape

(9653, 1121)

In [32]:
train_features.dtype

dtype('float64')

In [35]:
scaler_cqt = StandardScaler()
train_features_scaled = scaler_cqt.fit_transform(train_features)
val_features_scaled   = scaler_cqt.transform(val_features)
test_features_scaled  = scaler_cqt.transform(test_features)

In [36]:
start_time = time.time()

knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(train_features, train_labels)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_knn = knn_classifier.predict(val_features)
test_accuracy_knn = knn_classifier.predict(test_features)
print(f"KNN (FFT features) - Validation Accuracy: {accuracy_score(val_labels, val_pred_knn)*100:.2f}%")
print(f"KNN (FFT features) - Test Accuracy: {accuracy_score(test_labels, test_accuracy_knn)*100:.2f}%")

Training time: 0.02 seconds
KNN (FFT features) - Validation Accuracy: 85.35%
KNN (FFT features) - Test Accuracy: 86.16%


In [37]:
start_time = time.time()

knn_classifier_cqt = KNeighborsClassifier(n_neighbors=1, weights='uniform', algorithm='auto', leaf_size=30, p=2, 
                                      metric='minkowski', metric_params=None, n_jobs=None)

knn_classifier_cqt.fit(train_features_scaled, train_labels)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_cqt_knn = knn_classifier_cqt.predict(val_features_scaled)
test_pred_cqt_knn = knn_classifier_cqt.predict(test_features_scaled)

test_accuracy_cqt_knn = accuracy_score(test_labels, test_pred_cqt_knn)
print(f"KNN (CQT) - Test Accuracy: {test_accuracy_cqt_knn*100:.2f}%")


Training time: 0.01 seconds
KNN (CQT) - Test Accuracy: 83.95%


In [38]:
start_time = time.time()

svm_rbf_classifier = SVC(C=71.19418600172986, kernel='rbf', degree=3, gamma=0.03752055855124281, 
                         coef0=0.0, shrinking=True, probability=False, tol=0.0005319450186421158, 
                         cache_size=500, class_weight=None, verbose=False, max_iter=-1, 
                         decision_function_shape='ovr', break_ties=False, random_state=42)

# Kết hợp dữ liệu train và validation để tăng lượng dữ liệu huấn luyện (coi như là 8:2)
X_combined = np.vstack((train_features, val_features))
y_combined = np.concatenate((train_labels, val_labels))

# Huấn luyện mô hình SVM trên dữ liệu kết hợp
svm_rbf_classifier.fit(X_combined, y_combined)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ testing
test_predictions_svm_rbf = svm_rbf_classifier.predict(test_features)
test_accuracy_svm = accuracy_score(test_labels, test_predictions_svm_rbf)
print(f"Test Accuracy (SVM with RBF Kernel): {test_accuracy_svm * 100:.2f}%")

Training time: 4.89 seconds
Test Accuracy (SVM with RBF Kernel): 86.52%


In [39]:
start_time = time.time()

svm_rbf_classifier = SVC(C=71.19418600172986, kernel='rbf', degree=3, gamma=0.03752055855124281, 
                         coef0=0.0, shrinking=True, probability=False, tol=0.0005319450186421158, 
                         cache_size=500, class_weight=None, verbose=False, max_iter=-1, 
                         decision_function_shape='ovr', break_ties=False, random_state=42)

# Huấn luyện mô hình SVM
svm_rbf_classifier.fit(train_features_scaled, train_labels)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ validation
val_predictions_svm_rbf = svm_rbf_classifier.predict(val_features_scaled)
val_accuracy_svm_rbf = accuracy_score(val_labels, val_predictions_svm_rbf)
print(f"Validation Accuracy (SVM with RBF Kernel): {val_accuracy_svm_rbf * 100:.2f}%")

# Đánh giá mô hình trên bộ testing
test_predictions_svm_rbf = svm_rbf_classifier.predict(test_features_scaled)
scale_test_accuracy_svm = accuracy_score(test_labels, test_predictions_svm_rbf)
print(f"Test Accuracy (SVM with RBF Kernel): {scale_test_accuracy_svm * 100:.2f}%")

Training time: 4.17 seconds
Validation Accuracy (SVM with RBF Kernel): 84.55%
Test Accuracy (SVM with RBF Kernel): 85.69%


In [40]:
start_time = time.time()

lr_classifier = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=10, fit_intercept=True, intercept_scaling=1, 
                                   class_weight=None, random_state=42, solver='lbfgs', max_iter=1500, multi_class='deprecated', 
                                   verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
lr_classifier.fit(train_features, train_labels)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_lr = lr_classifier.predict(val_features)
test_accuracy_lr = lr_classifier.predict(test_features)
print(f"Logistic Regression (FFT features) - Validation Accuracy: {accuracy_score(val_labels, val_pred_lr)*100:.2f}%")
print(f"Logistic Regression (FFT features) - Test Accuracy: {accuracy_score(test_labels, test_accuracy_lr)*100:.2f}%")

Training time: 5.48 seconds
Logistic Regression (FFT features) - Validation Accuracy: 78.03%
Logistic Regression (FFT features) - Test Accuracy: 78.37%


C:\Users\dinhg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [41]:
start_time = time.time()

lr_classifier = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=10, fit_intercept=True, intercept_scaling=1, 
                                   class_weight=None, random_state=42, solver='saga', max_iter=1000, multi_class='deprecated', 
                                   verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

lr_classifier.fit(train_features_scaled, train_labels)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_lr = lr_classifier.predict(val_features_scaled)
scale_test_accuracy_lr = lr_classifier.predict(test_features_scaled)
print(f"Logistic Regression (FFT features) - Validation Accuracy: {accuracy_score(val_labels, val_pred_lr)*100:.2f}%")
print(f"Logistic Regression (FFT features) - Test Accuracy: {accuracy_score(test_labels, scale_test_accuracy_lr)*100:.2f}%")

Training time: 32.41 seconds
Logistic Regression (FFT features) - Validation Accuracy: 78.25%
Logistic Regression (FFT features) - Test Accuracy: 78.30%


C:\Users\dinhg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [42]:
start_time = time.time()

rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, 
                                       min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                       max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, 
                                       n_jobs=None, random_state=42, verbose=0, warm_start=False, class_weight=None, 
                                       ccp_alpha=0.0, max_samples=None, monotonic_cst=None)

rf_classifier.fit(train_features, train_labels)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

val_pred_rf = rf_classifier.predict(val_features)
test_accuracy_rf = rf_classifier.predict(test_features)
print(f"Random Forest (FFT features) - Validation Accuracy: {accuracy_score(val_labels, val_pred_rf)*100:.2f}%")
print(f"Random Forest (FFT features) - Test Accuracy: {accuracy_score(test_labels, test_accuracy_rf)*100:.2f}%")

Training time: 16.19 seconds
Random Forest (FFT features) - Validation Accuracy: 84.48%
Random Forest (FFT features) - Test Accuracy: 84.42%


In [43]:
start_time = time.time()

et_classifier = ExtraTreesClassifier(n_estimators=200, criterion='gini', max_depth=30, min_samples_split=2, min_samples_leaf=1, 
                                     min_weight_fraction_leaf=0.0, max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0,
                                     bootstrap=False, oob_score=False, n_jobs=None, random_state=42, verbose=0, warm_start=False, 
                                     class_weight=None, ccp_alpha=0.0, max_samples=None, monotonic_cst=None)

et_classifier.fit(train_features, train_labels)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ validation
val_predictions_et = et_classifier.predict(val_features)
val_accuracy_et = accuracy_score(val_labels, val_predictions_et)
print(f"Validation Accuracy (Extra Trees): {val_accuracy_et * 100:.2f}%")

# Đánh giá mô hình trên bộ testing
test_predictions_et = et_classifier.predict(test_features)
test_accuracy_et = accuracy_score(test_labels, test_predictions_et)
print(f"Test Accuracy (Extra Trees): {test_accuracy_et * 100:.2f}%")

Training time: 25.05 seconds
Validation Accuracy (Extra Trees): 86.80%
Test Accuracy (Extra Trees): 85.94%


In [44]:
start_time = time.time()

et_classifier = ExtraTreesClassifier(n_estimators=200, criterion='gini', max_depth=30, min_samples_split=2, min_samples_leaf=1, 
                                     min_weight_fraction_leaf=0.0, max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0,
                                     bootstrap=False, oob_score=False, n_jobs=None, random_state=42, verbose=0, warm_start=False, 
                                     class_weight=None, ccp_alpha=0.0, max_samples=None, monotonic_cst=None)

# Huấn luyện mô hình Extra Trees với dữ liệu đã chuẩn hóa
et_classifier.fit(train_features_scaled, train_labels)

end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

# Đánh giá mô hình trên bộ validation
val_predictions_et = et_classifier.predict(val_features_scaled)
val_accuracy_et = accuracy_score(val_labels, val_predictions_et)
print(f"Validation Accuracy (Extra Trees): {val_accuracy_et * 100:.2f}%")

# Đánh giá mô hình trên bộ testing
test_predictions_et = et_classifier.predict(test_features_scaled)
scale_test_accuracy_et = accuracy_score(test_labels, test_predictions_et)
print(f"Test Accuracy (Extra Trees): {scale_test_accuracy_et * 100:.2f}%")



Training time: 25.29 seconds
Validation Accuracy (Extra Trees): 86.87%
Test Accuracy (Extra Trees): 86.01%


In [45]:
from sklearn.ensemble import GradientBoostingClassifier

start_time = time.time()

gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, 
                            max_depth=3, min_samples_split=2,
                            min_samples_leaf=1, subsample=1.0,
                            max_features='sqrt', random_state=42)

gb_classifier.fit(train_features_scaled, train_labels)

end_time = time.time()
print(f"Training time (Gradient Boosting): {end_time - start_time:.2f} seconds")

val_predictions_gb = gb_classifier.predict(val_features_scaled)
val_accuracy_gb = accuracy_score(val_labels, val_predictions_gb)
print(f"Validation Accuracy (Gradient Boosting): {val_accuracy_gb * 100:.2f}%")

test_predictions_gb = gb_classifier.predict(test_features_scaled)
test_accuracy_gb = accuracy_score(test_labels, test_predictions_gb)
print(f"Test Accuracy (Gradient Boosting): {test_accuracy_gb * 100:.2f}%")
print("-" * 50)

Training time (Gradient Boosting): 15.12 seconds
Validation Accuracy (Gradient Boosting): 82.67%
Test Accuracy (Gradient Boosting): 83.66%
--------------------------------------------------


In [46]:
from sklearn.preprocessing import LabelEncoder
import time
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Chuyển đổi các nhãn từ chuỗi sang số
le = LabelEncoder()
train_labels_enc = le.fit_transform(train_labels)
val_labels_enc = le.transform(val_labels)
test_labels_enc = le.transform(test_labels)

start_time = time.time()

xgb_classifier = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=30,
    random_state=42,
    use_label_encoder=False,    # Tắt cảnh báo về label encoder
    eval_metric='logloss'       # Chỉ định hàm mất mát
)
xgb_classifier.fit(train_features, train_labels_enc)

end_time = time.time()
print(f"Training time (XGBoost): {end_time - start_time:.2f} seconds")

# Đánh giá trên bộ validation
val_predictions_xgb = xgb_classifier.predict(val_features)
val_accuracy_xgb = accuracy_score(val_labels_enc, val_predictions_xgb)
print(f"Validation Accuracy (XGBoost): {val_accuracy_xgb * 100:.2f}%")

# Đánh giá trên bộ testing
test_predictions_xgb = xgb_classifier.predict(test_features)
test_accuracy_xgb = accuracy_score(test_labels_enc, test_predictions_xgb)
print(f"Test Accuracy (XGBoost): {test_accuracy_xgb * 100:.2f}%")


C:\Users\dinhg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\core.py:158: UserWarning: [19:04:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Training time (XGBoost): 36.81 seconds
Validation Accuracy (XGBoost): 84.84%
Test Accuracy (XGBoost): 86.05%


In [47]:
import lightgbm as lgb
import time
from sklearn.metrics import accuracy_score

start_time = time.time()

lgbm_classifier = lgb.LGBMClassifier(
    n_estimators=200,
    max_depth=30,
    random_state=42
)
lgbm_classifier.fit(train_features, train_labels)

end_time = time.time()
print(f"\nTraining time (LightGBM): {end_time - start_time:.2f} seconds")

# Đánh giá trên bộ validation
val_predictions_lgbm = lgbm_classifier.predict(val_features)
val_accuracy_lgbm = accuracy_score(val_labels, val_predictions_lgbm)
print(f"Validation Accuracy (LightGBM): {val_accuracy_lgbm * 100:.2f}%")

# Đánh giá trên bộ testing
test_predictions_lgbm = lgbm_classifier.predict(test_features)
test_accuracy_lgbm = accuracy_score(test_labels, test_predictions_lgbm)
print(f"Test Accuracy (LightGBM): {test_accuracy_lgbm * 100:.2f}%")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017041 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 9653, number of used features: 100
[LightGBM] [Info] Start training from score -0.924143
[LightGBM] [Info] Start training from score -1.383501
[LightGBM] [Info] Start training from score -1.042905

Training time (LightGBM): 1.74 seconds
Validation Accuracy (LightGBM): 84.70%
Test Accuracy (LightGBM): 86.78%
